In [ ]:
import time
import picamera
import picamera.array
import modules.candle as candle

camera = picamera.PiCamera()
time.sleep(1)
stream = picamera.array.PiRGBArray(camera)

from BrickPi import *
BrickPiSetup()
BrickPi.MotorEnable[PORT_A] = 1
BrickPi.MotorEnable[PORT_D] = 1

In [ ]:
stream.truncate(0)
camera.capture(stream, format='rgb')
image = stream.array
imshow(image)

found,x,y,w,h = candle.position(image)
if found:
    print('Candle bounding box: (x,y)=(%d,%d), w=%d, h=%d ' % (x,y,w,h))
    plot([x,x+w,x+w,x,x],[y,y,y+h,y+h,y],'y-')
    lit = candle.state(image,x,y,w,h)
    if lit:
        print('It seems to be ON')
    else:
        print('It seems to be OFF')
else:
    print('Candle not found')


In [ ]:
from IPython.display import clear_output
try:
    converged = False
    while not converged:
        stream.truncate(0)
        camera.capture(stream, format='rgb')
        image = stream.array
        found,x,y,w,h = candle.position(image)
        clear_output()
        if found:
            xm = x + w/2
            dx = xm - 360
            motor = dx * 500 / 360
            if motor > 50:
                motor = 50
            if motor < -50:
                motor = -50
            print(xm,dx,motor)
            BrickPi.MotorSpeed[PORT_A] = motor
            BrickPi.MotorSpeed[PORT_D] = -motor
            BrickPiUpdateValues()
            time.sleep(0.1)
            BrickPi.MotorSpeed[PORT_A] = 0
            BrickPi.MotorSpeed[PORT_D] = 0
            BrickPiUpdateValues()
            converged = abs(motor) < 10
        else:
            print("candle not found")
except KeyboardInterrupt:
    pass

<hr>
<center>
    <p>&copy; Enric Cervera 2016</p>
    <p><a href="http://www.uji.es"><img alt="Universitat Jaume I" src="files/img/uji_logo.jpg" /></a></p>
    <p><a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.</p>
</center>